# Chapter 05-01: CNN 기초 — 합성곱 신경망

## 학습 목표
- 합성곱(Convolution) 연산의 수학적 원리를 이해한다
- Padding과 Stride가 출력 크기에 미치는 영향을 계산할 수 있다
- Pooling 레이어의 종류와 역할을 설명할 수 있다
- 기본 CNN 구조를 TensorFlow/Keras로 구현하고 MNIST를 학습시킬 수 있다
- 중간 레이어의 특징 맵(Feature Map)을 시각화할 수 있다

## 목차
1. [합성곱 연산 수학 공식](#1)
2. [Numpy로 수동 2D 합성곱 구현](#2)
3. [Padding과 Stride](#3)
4. [Pooling 레이어 비교](#4)
5. [기본 CNN 구성 및 MNIST 학습](#5)
6. [특징 맵 시각화](#6)
7. [정리](#7)

In [ ]:
# 필수 라이브러리 임포트
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

# 한글 폰트 설정 (macOS 기준)
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False

# 재현성을 위한 시드 고정
tf.random.set_seed(42)
np.random.seed(42)

print(f'TensorFlow 버전: {tf.__version__}')
print(f'NumPy 버전: {np.__version__}')

## 1. 합성곱 연산 수학 공식 <a id='1'></a>

### 2D 합성곱 (2D Convolution)

입력 이미지 $I$와 커널 $K$의 합성곱은 다음과 같이 정의된다:

$(I * K)[i,j] = \sum_m \sum_n I[i+m,\, j+n] \cdot K[m,n]$

여기서 $i, j$는 출력의 위치, $m, n$은 커널 내 상대적 위치이다.

### 출력 크기 계산

입력 크기 $I$, 커널 크기 $K$, 패딩 $P$, 스트라이드 $S$가 주어질 때 출력 크기 $O$는:

$O = \lfloor\frac{I - K + 2P}{S}\rfloor + 1$

예시: 입력 $28 \times 28$, 커널 $3 \times 3$, 패딩 0, 스트라이드 1이면
$O = \lfloor\frac{28 - 3 + 0}{1}\rfloor + 1 = 26$

### Conv2D 파라미터 수

커널 높이 $K_h$, 커널 너비 $K_w$, 입력 채널 $C_{in}$, 출력 채널 $C_{out}$일 때:

$\text{파라미터 수} = K_h \times K_w \times C_{in} \times C_{out} + C_{out}$

마지막 $+ C_{out}$은 편향(bias) 항이다.

## 2. Numpy로 수동 2D 합성곱 구현 <a id='2'></a>

합성곱 연산의 직관을 얻기 위해 NumPy만으로 2D 합성곱을 직접 구현해 본다.
엣지 검출(Edge Detection) 커널로 실제 이미지에 적용한다.

대표적인 엣지 검출 커널:
- **Sobel X**: 수직 방향 엣지 검출
- **Sobel Y**: 수평 방향 엣지 검출
- **Laplacian**: 전방향 엣지 검출

In [ ]:
def conv2d_manual(image, kernel, stride=1, padding=0):
    """
    NumPy로 구현한 수동 2D 합성곱 함수
    
    Args:
        image: 2D 입력 이미지 배열 (H, W)
        kernel: 2D 커널 배열 (Kh, Kw)
        stride: 스트라이드 (기본값 1)
        padding: 제로 패딩 크기 (기본값 0)
    
    Returns:
        출력 특징 맵 (output feature map)
    """
    # 패딩 적용
    if padding > 0:
        image = np.pad(image, padding, mode='constant', constant_values=0)
    
    I_h, I_w = image.shape          # 입력 높이, 너비
    K_h, K_w = kernel.shape         # 커널 높이, 너비
    
    # 출력 크기 계산: O = floor((I - K) / S) + 1
    O_h = (I_h - K_h) // stride + 1
    O_w = (I_w - K_w) // stride + 1
    
    # 출력 배열 초기화
    output = np.zeros((O_h, O_w))
    
    # 합성곱 연산: 슬라이딩 윈도우 방식
    for i in range(O_h):
        for j in range(O_w):
            # 현재 윈도우 위치에서 요소별 곱셈 후 합산
            output[i, j] = np.sum(
                image[i*stride:i*stride+K_h, j*stride:j*stride+K_w] * kernel
            )
    
    return output


# 테스트용 간단한 이미지 생성 (6x6)
test_image = np.array([
    [0, 0, 0, 0, 0, 0],
    [0, 1, 1, 1, 1, 0],
    [0, 1, 2, 2, 1, 0],
    [0, 1, 2, 2, 1, 0],
    [0, 1, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 0]
], dtype=float)

# 엣지 검출 커널 정의
sobel_x = np.array([[-1, 0, 1],
                     [-2, 0, 2],
                     [-1, 0, 1]], dtype=float)  # 수직 엣지 검출

sobel_y = np.array([[-1, -2, -1],
                     [ 0,  0,  0],
                     [ 1,  2,  1]], dtype=float)  # 수평 엣지 검출

laplacian = np.array([[ 0, -1,  0],
                       [-1,  4, -1],
                       [ 0, -1,  0]], dtype=float)  # 전방향 엣지 검출

# 각 커널 적용
out_sobel_x = conv2d_manual(test_image, sobel_x)
out_sobel_y = conv2d_manual(test_image, sobel_y)
out_laplacian = conv2d_manual(test_image, laplacian)

# 시각화
fig, axes = plt.subplots(1, 4, figsize=(14, 3))

axes[0].imshow(test_image, cmap='gray')
axes[0].set_title('원본 이미지')

axes[1].imshow(out_sobel_x, cmap='RdBu')
axes[1].set_title('Sobel X (수직 엣지)')

axes[2].imshow(out_sobel_y, cmap='RdBu')
axes[2].set_title('Sobel Y (수평 엣지)')

axes[3].imshow(out_laplacian, cmap='RdBu')
axes[3].set_title('Laplacian (전방향 엣지)')

for ax in axes:
    ax.axis('off')

plt.suptitle('엣지 검출 커널 적용 결과', fontsize=14)
plt.tight_layout()
plt.show()

print(f'입력 크기: {test_image.shape}')
print(f'출력 크기 (3x3 커널, stride=1, padding=0): {out_sobel_x.shape}')

## 3. Padding과 Stride <a id='3'></a>

### Padding
- **valid**: 패딩 없음. 입력 경계 외부는 무시 → 출력 크기 감소
- **same**: 출력 크기 = 입력 크기가 되도록 자동으로 제로 패딩 추가

### Stride
- 커널이 이동하는 간격
- stride=2이면 출력 크기가 약 절반으로 줄어든다
- MaxPooling 없이 Stride로 다운샘플링하는 경우도 있다

In [ ]:
# Padding과 Stride 효과 비교

# 28x28 더미 입력 (MNIST 크기)
dummy_input = tf.random.normal([1, 28, 28, 1])

# 다양한 설정으로 Conv2D 적용
configs = [
    {'filters': 32, 'kernel_size': 3, 'strides': 1, 'padding': 'valid', 'label': 'valid, stride=1'},
    {'filters': 32, 'kernel_size': 3, 'strides': 1, 'padding': 'same',  'label': 'same,  stride=1'},
    {'filters': 32, 'kernel_size': 3, 'strides': 2, 'padding': 'valid', 'label': 'valid, stride=2'},
    {'filters': 32, 'kernel_size': 3, 'strides': 2, 'padding': 'same',  'label': 'same,  stride=2'},
]

print(f'{'설정':<25} {'입력 크기':<15} {'출력 크기':<15} {'이론값'}')
print('-' * 70)

for cfg in configs:
    layer = tf.keras.layers.Conv2D(
        filters=cfg['filters'],
        kernel_size=cfg['kernel_size'],
        strides=cfg['strides'],
        padding=cfg['padding']
    )
    output = layer(dummy_input)
    
    # 이론적 출력 크기 계산 (valid 패딩 기준)
    if cfg['padding'] == 'valid':
        theory = (28 - cfg['kernel_size']) // cfg['strides'] + 1
    else:  # same
        theory = int(np.ceil(28 / cfg['strides']))
    
    print(f"{cfg['label']:<25} {str(dummy_input.shape[1:3]):<15} "
          f"{str(output.shape[1:3]):<15} H={theory}")

## 4. Pooling 레이어 비교 <a id='4'></a>

Pooling은 공간 차원을 줄여 계산량을 감소시키고 과적합을 방지하는 역할을 한다.

| 종류 | 설명 | 특징 |
|------|------|------|
| MaxPooling2D | 윈도우 내 최댓값 선택 | 가장 강한 특징 보존 |
| AveragePooling2D | 윈도우 내 평균값 | 부드러운 특징 추출 |
| GlobalAveragePooling2D | 채널별 전체 평균 | Flatten 대체, 파라미터 절약 |

In [ ]:
# Pooling 레이어 비교 예시

# 4x4 특징 맵 예시
feature_map = np.array([[[[1, 2, 3, 4],
                           [5, 6, 7, 8],
                           [9, 10, 11, 12],
                           [13, 14, 15, 16]]]], dtype=float)
# 형태 변환: (1, 4, 4, 1)
feature_map = feature_map.transpose(0, 2, 3, 1)
print(f'입력 특징 맵 형태: {feature_map.shape}')
print(f'입력:\n{feature_map[0,:,:,0]}')
print()

# MaxPooling2D (2x2, stride=2)
max_pool = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2)
out_max = max_pool(feature_map)
print(f'MaxPooling2D 출력 (2x2, stride=2):\n{out_max[0,:,:,0].numpy()}')
print()

# AveragePooling2D (2x2, stride=2)
avg_pool = tf.keras.layers.AveragePooling2D(pool_size=2, strides=2)
out_avg = avg_pool(feature_map)
print(f'AveragePooling2D 출력 (2x2, stride=2):\n{out_avg[0,:,:,0].numpy()}')
print()

# GlobalAveragePooling2D (채널별 전체 평균 → 1D 벡터)
gap = tf.keras.layers.GlobalAveragePooling2D()
out_gap = gap(feature_map)
print(f'GlobalAveragePooling2D 출력 (채널별 평균): {out_gap.numpy()}')
print(f'  → 전체 평균: {feature_map.mean():.1f} (확인)')
print()

# 시각화
fig, axes = plt.subplots(1, 3, figsize=(12, 3))

axes[0].imshow(out_max[0,:,:,0], cmap='Blues', vmin=1, vmax=16)
for i in range(2):
    for j in range(2):
        axes[0].text(j, i, f'{out_max[0,i,j,0].numpy():.0f}',
                     ha='center', va='center', fontsize=16, fontweight='bold')
axes[0].set_title('MaxPooling2D\n(최댓값 선택)')
axes[0].axis('off')

axes[1].imshow(out_avg[0,:,:,0], cmap='Greens', vmin=1, vmax=16)
for i in range(2):
    for j in range(2):
        axes[1].text(j, i, f'{out_avg[0,i,j,0].numpy():.1f}',
                     ha='center', va='center', fontsize=16, fontweight='bold')
axes[1].set_title('AveragePooling2D\n(평균값)')
axes[1].axis('off')

axes[2].imshow([[out_gap[0,0].numpy()]], cmap='Oranges', vmin=1, vmax=16)
axes[2].text(0, 0, f'{out_gap[0,0].numpy():.1f}',
             ha='center', va='center', fontsize=20, fontweight='bold')
axes[2].set_title('GlobalAveragePooling2D\n(전체 평균 → 스칼라)')
axes[2].axis('off')

plt.suptitle('Pooling 레이어 비교', fontsize=14)
plt.tight_layout()
plt.show()

## 5. 기본 CNN 구성 및 MNIST 학습 <a id='5'></a>

전형적인 CNN 구조:
```
Conv2D → MaxPool → Conv2D → MaxPool → Flatten → Dense → Dense(출력)
```

각 단계의 역할:
- **Conv2D + ReLU**: 지역적 패턴 (엣지, 텍스처, 형태) 추출
- **MaxPooling**: 공간 해상도 축소, 평행이동 불변성 부여
- **Flatten**: 2D 특징 맵을 1D 벡터로 변환
- **Dense**: 전역적 특징 조합 및 분류

In [ ]:
# MNIST 데이터 로드 및 전처리
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# CNN 입력을 위해 채널 차원 추가 및 정규화 [0, 255] → [0, 1]
x_train = x_train[..., np.newaxis] / 255.0  # (60000, 28, 28, 1)
x_test  = x_test[..., np.newaxis]  / 255.0  # (10000, 28, 28, 1)

print(f'학습 데이터: {x_train.shape}, 레이블: {y_train.shape}')
print(f'테스트 데이터: {x_test.shape}, 레이블: {y_test.shape}')

# 기본 CNN 모델 구성
model = tf.keras.Sequential([
    # 첫 번째 합성곱 블록
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu',
                           input_shape=(28, 28, 1), name='conv1'),
    tf.keras.layers.MaxPooling2D((2, 2), name='pool1'),
    
    # 두 번째 합성곱 블록
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', name='conv2'),
    tf.keras.layers.MaxPooling2D((2, 2), name='pool2'),
    
    # 분류 헤드
    tf.keras.layers.Flatten(name='flatten'),
    tf.keras.layers.Dense(128, activation='relu', name='dense1'),
    tf.keras.layers.Dropout(0.3, name='dropout'),
    tf.keras.layers.Dense(10, activation='softmax', name='output')
], name='basic_cnn')

model.summary()

# 모델 컴파일
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# 모델 학습
history = model.fit(
    x_train, y_train,
    epochs=5,
    batch_size=128,
    validation_split=0.1,
    verbose=1
)

# 테스트 세트 평가
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f'\n테스트 정확도: {test_acc:.4f}')

# 학습 곡선 시각화
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot(history.history['loss'], label='학습 손실')
ax1.plot(history.history['val_loss'], label='검증 손실')
ax1.set_xlabel('에폭')
ax1.set_ylabel('손실')
ax1.set_title('학습/검증 손실 곡선')
ax1.legend()
ax1.grid(True)

ax2.plot(history.history['accuracy'], label='학습 정확도')
ax2.plot(history.history['val_accuracy'], label='검증 정확도')
ax2.set_xlabel('에폭')
ax2.set_ylabel('정확도')
ax2.set_title('학습/검증 정확도 곡선')
ax2.legend()
ax2.grid(True)

plt.suptitle(f'Basic CNN on MNIST (테스트 정확도: {test_acc:.4f})', fontsize=13)
plt.tight_layout()
plt.show()

## 6. 특징 맵 시각화 <a id='6'></a>

학습된 CNN의 중간 레이어 출력을 추출하여 각 필터가 어떤 특징을 감지하는지 확인한다.

**방법**: `tf.keras.Model`을 이용해 중간 레이어의 출력을 추출하는 서브모델 생성

In [ ]:
# 중간 레이어 출력을 추출하는 시각화 모델 생성
layer_names = ['conv1', 'pool1', 'conv2', 'pool2']

# 각 레이어의 출력을 반환하는 서브모델
visualization_model = tf.keras.Model(
    inputs=model.input,
    outputs=[model.get_layer(name).output for name in layer_names]
)

# 샘플 이미지 선택 (숫자 '7')
sample_idx = np.where(y_test == 7)[0][0]
sample_image = x_test[sample_idx:sample_idx+1]  # (1, 28, 28, 1)

# 중간 레이어 출력 계산
feature_maps = visualization_model.predict(sample_image, verbose=0)

# 각 레이어의 특징 맵 시각화
fig = plt.figure(figsize=(16, 10))

# 원본 이미지
ax = fig.add_subplot(5, 1, 1)
ax.imshow(sample_image[0, :, :, 0], cmap='gray')
ax.set_title(f'원본 이미지 (레이블: {y_test[sample_idx]})', fontsize=12)
ax.axis('off')

# 각 레이어의 특징 맵 (최대 16개 필터)
n_display = 16  # 표시할 필터 수

for layer_idx, (layer_name, fmap) in enumerate(zip(layer_names, feature_maps)):
    n_filters = min(fmap.shape[-1], n_display)
    
    # 서브플롯 그리드 (4열)
    for f_idx in range(n_filters):
        ax = fig.add_subplot(5, n_display, (layer_idx + 1) * n_display + f_idx + 1)
        ax.imshow(fmap[0, :, :, f_idx], cmap='viridis')
        ax.axis('off')
        if f_idx == 0:
            ax.set_ylabel(f'{layer_name}\n{fmap.shape[1:3]}', fontsize=9)

plt.suptitle('CNN 중간 레이어 특징 맵 시각화', fontsize=14, y=0.98)
plt.tight_layout()
plt.show()

# 각 레이어별 정보 출력
print('\n레이어별 특징 맵 정보:')
print(f'{'레이어':<15} {'출력 형태':<20} {'파라미터 수'}')
print('-' * 50)
for layer_name, fmap in zip(layer_names, feature_maps):
    layer = model.get_layer(layer_name)
    params = layer.count_params()
    print(f'{layer_name:<15} {str(fmap.shape[1:]):<20} {params:,}')

## 7. 정리 <a id='7'></a>

### 핵심 수식 요약

| 항목 | 수식 |
|------|------|
| 2D 합성곱 | $(I * K)[i,j] = \sum_m \sum_n I[i+m, j+n] \cdot K[m,n]$ |
| 출력 크기 | $O = \lfloor\frac{I - K + 2P}{S}\rfloor + 1$ |
| Conv2D 파라미터 | $K_h \times K_w \times C_{in} \times C_{out} + C_{out}$ |

### 핵심 개념
- **합성곱**: 슬라이딩 윈도우로 지역 패턴을 추출하는 연산
- **패딩(same)**: 출력 크기를 입력과 동일하게 유지
- **스트라이드**: 클수록 출력 크기 감소 (다운샘플링)
- **MaxPooling**: 공간 크기 축소 + 평행이동 불변성
- **특징 맵**: 각 필터가 감지하는 특정 패턴의 활성화 강도

### 다음 챕터 예고
**Chapter 05-02**: VGG, ResNet, EfficientNet 등 실제 서비스에 사용되는 고급 CNN 아키텍처를 학습한다.